In [ ]:
from os.path import join, dirname
from dotenv import load_dotenv
import os
import pickle
from snowflake import connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from scipy.optimize import curve_fit
from sklearn.preprocessing import Normalizer, QuantileTransformer, RobustScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error, make_scorer


pd.options.display.float_format = '{:,.2f}'.format

# get environment variables
dotenv_path = join(dirname('streamlit_grs_fit\\app\\'), '.env')
load_dotenv(dotenv_path)
SF_ACCOUNT = os.getenv('SF_ACCOUNT')
SF_USER = os.getenv('SF_USER')
SF_PASSWORD = os.getenv('SF_PASSWORD')
SF_ROLE = os.getenv('SF_ROLE')
SF_WAREHOUSE = os.getenv('SF_WAREHOUSE')
SF_DATABASE = os.getenv('SF_DATABASE')
SF_SCHEMA = os.getenv('SF_SCHEMA')

def load_data(query):
    conn = connector.connect(
        user = SF_USER
        ,password = SF_PASSWORD
        ,account = SF_ACCOUNT
        ,warehouse = SF_WAREHOUSE
        ,database = SF_DATABASE
        ,schema = SF_SCHEMA
        ,role = SF_ROLE
    )
    cur = conn.cursor()
    df_data = cur.execute(query).fetch_pandas_all()
    return df_data

In [ ]:
query = 'select '+\
            'JOB'+\
            ',DIRECT_COST'+\
            ',DIV_00_DIRECT_COST'+\
            ',DIV_01_DIRECT_COST'+\
            ',DIV_02_DIRECT_COST'+\
            ',DIV_03_DIRECT_COST'+\
            ',DIV_04_DIRECT_COST'+\
            ',DIV_05_DIRECT_COST'+\
            ',DIV_06_DIRECT_COST'+\
            ',DIV_07_DIRECT_COST'+\
            ',DIV_08_DIRECT_COST'+\
            ',DIV_09_DIRECT_COST'+\
            ',DIV_10_DIRECT_COST'+\
            ',DIV_11_DIRECT_COST'+\
            ',DIV_12_DIRECT_COST'+\
            ',DIV_13_DIRECT_COST'+\
            ',DIV_14_DIRECT_COST'+\
            ',DIV_15_DIRECT_COST'+\
            ',DIV_16_DIRECT_COST'+\
            ',DIV_17_DIRECT_COST'+\
            ',DIV_18_DIRECT_COST'+\
            ',DIV_19_DIRECT_COST'+\
            ',DIV_21_DIRECT_COST'+\
            ',DIV_22_DIRECT_COST'+\
            ',DIV_23_DIRECT_COST'+\
            ',DIV_26_DIRECT_COST'+\
            ',DIV_27_DIRECT_COST'+\
            ',DIV_28_DIRECT_COST'+\
            ',DIV_31_DIRECT_COST'+\
            ',DIV_32_DIRECT_COST'+\
            ',DIV_33_DIRECT_COST'+\
            ',DIV_34_DIRECT_COST'+\
            ',DIV_55_DIRECT_COST'+\
            ',GCS_COST'+\
            ',GRS_COST '+\
            'from sandbox.global.ml_grs_fit ' 
df_data = load_data(query).set_index('JOB') 
df_data = pd.DataFrame(df_data)
df_data = df_data.fillna(0)

In [ ]:
df_working = df_data.loc[
                    (0 != df_data.GRS_COST) &
                    (0 != df_data.GCS_COST)
].copy()
df_working.describe()

In [ ]:
X  = df_working.iloc[:,:-2] #.values
y_gcs = df_working.iloc[:,-2:-1].values.ravel()
y_grs = df_working.iloc[:,-1:].values.ravel()
y_grs.shape

In the last itteration, I got everything I was trying to do to work. that is, I trained a model and saved it as a binary object that can be packaged with an app, and tested the implementation within the app framework. 
that part all works great! it's super fast, and I think we'll be able to get the ui pretty nice at the end.
the thing to be aware of at this point is that it can't be expected to be reliable. yet
here's what I think are next steps in order to make it as reliable as possible, and to be able to know exactly how reliable it is. this is what I'll do in this itteration:
1.  identify metrics by which model success will be measured. e.g., MSE, RMSE, MAE, MAPE
2.  implement train / test split prior to cross validation
3.  rerun the cross validation setup for this estimator (knearestneighborsregressor) with the training data only
4.  evaluate, in terms of our selected metrics, the model selected by cross validation against the test data (to which it would be naïve at this point. this will guard us against data leakage)
5.  repeat steps 3 and 4 for a few other estimators. I'm thinking lasso, elasticnet, maybe ridgeregression or svr, and perhaps mlpregressor if we're feeling ambitious
compare the scores of our models and pick the best one

initially, trying these scorers: r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error

In [ ]:
X_train, X_test, y_grs_train, y_grs_test = train_test_split(X, y_grs, test_size=0.33, random_state=42)
X_train, X_test, y_gcs_train, y_gcs_test = train_test_split(X, y_gcs, test_size=0.33, random_state=42)

In [ ]:
def print_dataframe(filtered_cv_results):
    """Pretty print for filtered dataframe"""
    for mean_r2, std_r2, mean_negmedae, std_negmedae, params in zip(
        filtered_cv_results["mean_test_R2"],
        filtered_cv_results["std_test_R2"],
        filtered_cv_results["mean_test_negMedAE"],
        filtered_cv_results["std_test_negMedAE"],
        filtered_cv_results["params"],
    ):
        print(
            f"R2: {mean_r2:0.3f} (±{std_r2:0.03f}),"
            f" negMedAE: {mean_negmedae:0.3f} (±{std_negmedae:0.03f}),"
            f" for {params}"
        )
    print()

def refit_strategy(cv_results):
    """Define the strategy to select the best estimator.

    The strategy defined here is to filter-out all results below a R2 threshold
    of 0.5, rank the remaining by MedAE and keep all models with one standard
    deviation of the best by MedAE. Once these models are selected, we can select the
    fastest model to predict.

    Parameters
    ----------
    cv_results : dict of numpy (masked) ndarrays
        CV results as returned by the `GridSearchCV`.

    Returns
    -------
    best_index : int
        The index of the best estimator as it appears in `cv_results`.
    """
    # print the info about the grid-search for the different scores
    r2_threshold = 0.67

    cv_results_ = pd.DataFrame(cv_results)
    print("All grid-search results:")
    print_dataframe(cv_results_)

    # Filter-out all results below the threshold
    high_r2_cv_results = cv_results_[
        cv_results_["mean_test_R2"] > r2_threshold
    ]

    print(f"Models with an R2 higher than {r2_threshold}:")
    print_dataframe(high_r2_cv_results)

    high_r2_cv_results = high_r2_cv_results[
        [
            "mean_score_time",
            "mean_test_R2",
            "std_test_R2",
            "mean_test_negMedAE",
            "std_test_negMedAE",
            "rank_test_R2",
            "rank_test_negMedAE",
            "params",
        ]
    ]

    # Select the most performant models in terms of negMedAE
    # (within 1 sigma from the best)
    best_negmedae_std = high_r2_cv_results["mean_test_negMedAE"].std()
    best_negmedae = high_r2_cv_results["mean_test_negMedAE"].max()
    best_negmedae_threshold = best_negmedae - best_negmedae_std

    high_negmedae_cv_results = high_r2_cv_results[
        high_r2_cv_results["mean_test_negMedAE"] > best_negmedae_threshold
    ]
    print(
        "Out of the previously selected high R2 models, we keep all the\n"
        "the models within one standard deviation of the highest negMedAE model:"
    )
    print_dataframe(high_negmedae_cv_results)

    # From the best candidates, select the fastest model to predict
    fastest_top_negmedae_high_r2_index = high_negmedae_cv_results[
        "mean_score_time"
    ].idxmin()

    print(
        "\nThe selected final model is the fastest to predict out of the previously\n"
        "selected subset of best models based on R2 and negMedAE.\n"
        "Its scoring time is:\n\n"
        f"{high_negmedae_cv_results.loc[fastest_top_negmedae_high_r2_index]}"
    )

    return fastest_top_negmedae_high_r2_index

In [ ]:
# make a custom scorer
def neg_median_absolute_error(y_true, y_pred):
    medae = median_absolute_error(y_true, y_pred)
    return -medae

# pipeline setup
pipeline = Pipeline([
                     ('scaler', RobustScaler())            #start with this scaler. scalers account for differences in scale between features
                    ,('kbest', SelectKBest(f_classif))     #select k best indy vars where k = winning parameter below
                     ,('regressor', Lasso())                #the estimator, aka the ai
                     ])

parameters = {
                'scaler':  [RobustScaler(), Normalizer(), QuantileTransformer()]   #try all these scalers
                ,'kbest__k':  list(range(1, X.shape[1]+1))                         #with all these numbers of best indy vars (between 1 and the number of vars)
                ,'regressor__alpha':[0.005, 0.02, 0.03, 0.05, 0.06]                #try various alpha settings
                }
#grs model
grs_grid = GridSearchCV(
    pipeline
    ,parameters
    ,cv=10
    ,scoring={'R2': make_scorer(r2_score)
            ,'negMedAE': make_scorer(neg_median_absolute_error)
    }
    # ,refit='R2'
    ,refit=refit_strategy
    ,return_train_score=False
    ,n_jobs=-1
)   

#gcs model
gcs_grid = GridSearchCV(
    pipeline
    ,parameters
    ,cv=10
    ,scoring={'R2': make_scorer(r2_score)
            ,'negMedAE': make_scorer(neg_median_absolute_error)
    }
    ,refit=refit_strategy
    ,return_train_score=False
    ,n_jobs=-1
)   

In [ ]:
grs_grid = grs_grid.fit(X_train, y_grs_train)

In [ ]:
gcs_grid = gcs_grid.fit(X_train, y_gcs_train) #fails to converge, therefore knnr is superior

In [ ]:
y_grs_test_pred = grs_grid.best_estimator_.predict(X_test)
print(f'R2 for test grs data: {r2_score(y_grs_test, y_grs_test_pred)}')
print(f'negMedAE for test grs data: {neg_median_absolute_error(y_grs_test, y_grs_test_pred)}')

In [ ]:
y_gcs_test_pred = gcs_grid.best_estimator_.predict(X_test)
print(f'R2 for test gcs data: {r2_score(y_gcs_test, y_gcs_test_pred)}')
print(f'negMedAE for test gcs data: {neg_median_absolute_error(y_gcs_test, y_gcs_test_pred)}')

In [ ]:
gcs_grid_test = gcs_grid.best_estimator_.fit(X_test, y_gcs_test)

In [ ]:
print("the best grs estimator is \n {} ".format(grs_grid.best_estimator_))
print("the best grs parameters are \n {}".format(grs_grid.best_params_))
print("the best gcs estimator is \n {} ".format(gcs_grid.best_estimator_))
print("the best gcs parameters are \n {}".format(gcs_grid.best_params_))

In [ ]:
grs_best_pipe = grs_grid.best_estimator_
grs_mask = list(grs_best_pipe.fit(X,y_grs)[:-1].get_feature_names_out())
grs_model = grs_best_pipe.fit(df_working[grs_mask],y_grs)
grs_predictions = grs_model.predict(df_working[grs_mask])

In [ ]:
gcs_best_pipe = gcs_grid.best_estimator_
gcs_mask = list(gcs_best_pipe.fit(X,y_gcs)[:-1].get_feature_names_out())
gcs_model = gcs_best_pipe.fit(df_working[gcs_mask],y_gcs)
gcs_predictions = gcs_model.predict(df_working[gcs_mask])

In [ ]:
list(grs_model[:-1].get_feature_names_out())

In [ ]:
grs_parameters = list(df_working[grs_mask].columns)
gcs_parameters = list(df_working[gcs_mask].columns)
combined_mask = list(set(grs_parameters + gcs_parameters))
df = df_working[combined_mask].copy()
df['GRS_PREDICTIONS'] = grs_predictions
df['GCS_PREDICTIONS'] = gcs_predictions
knnr_model_bag = {
    'df': df
    ,'grs_model': grs_model
    ,'grs_parameters': grs_parameters
    ,'gcs_model': gcs_model
    ,'gcs_parameters': gcs_parameters
}
with open('./app/knnr_model_bag.pkl','wb') as p:
    pickle.dump(knnr_model_bag, p, protocol=-1)

In [ ]:
with open('./app/knnr_model_bag.pkl','rb') as p:
    bag = pickle.load(p)

In [ ]:
bag.keys()

In [ ]:
grs_params = bag['grs_parameters']
gcs_params = bag['gcs_parameters']
all_params = list(set(grs_params + gcs_params))
test_vec = bag['df'][all_params].sample(1).copy()
# bag['grs_model'].predict(test_vec)
# model = bag['grs_model']
# list(model[:-1].get_feature_names_out())
# print(*list(test_vec[grs_params].columns), sep='\n,')
gcs_params

In [ ]:
vec = test_vec.reset_index(drop=True).T
vec.index.names = ['PARAMETERS']
vec = vec.reset_index()
vec.set_index('PARAMETERS').sort_index()

In [ ]:
bag['grs_model'].predict(test_vec)

In [ ]:
# get the features scores rounded in 2 decimals
pip_steps = grs_grid.best_estimator_.named_steps['kbest']
pip_steps.get_support()

features_scores = ['%.2f' % elem for elem in pip_steps.scores_ ]
print("the features scores are \n {}".format(features_scores))

feature_scores_pvalues = ['%.3f' % elem for elem in pip_steps.pvalues_]
print("the feature_pvalues is \n {} ".format(feature_scores_pvalues))

scored_features = pd.DataFrame(df_working[grs_mask].columns, columns=['feature_names'])
scored_features['feature_scores'] = features_scores
scored_features['feature_scores_pvalues'] = feature_scores_pvalues
scored_features = scored_features.loc[(scored_features['feature_scores'] != 'nan') & (scored_features['feature_scores'] != 'inf')].sort_values(by='feature_scores', ascending=False).iloc[:num_features]
scored_features

In [ ]:
selected_features = scored_features.feature_names.to_list()
df_working[selected_features].describe()

In [ ]:
pickle.dump(neigh, open('grs_model.pkl','wb'))

In [ ]:
grs_model = pickle.load(open('grs_model.pkl','rb'))

In [ ]:
pickle.dump(data_preds, open('grs_model.pkl','ab+'))

In [ ]:
grs_data = []
with open('./app/grs_model.pkl', 'rb') as fr:
    try:
        while True:
            grs_data.append(pickle.load(fr))
    except EOFError:
        pass
gcs_data = []
with open('./app/gcs_model.pkl', 'rb') as fr:
    try:
        while True:
            gcs_data.append(pickle.load(fr))
    except EOFError:
        pass

In [ ]:
grs_model, grs_preds = grs_data
gcs_model, gcs_preds = gcs_data

In [ ]:
graphWidth = 1500
graphHeight = graphWidth * 800 / 1000
x_plot = data_preds.DIRECT_COST
y1_plot = data_preds.GRS_ACTUAL
y2_plot = data_preds.GRS_PREDICTIONS
f = plt.figure(figsize=(graphWidth/100.0, graphHeight/100.0), dpi=100)
axes = f.add_subplot(111)
axes.plot(x_plot, y1_plot, c='g', alpha=0.15)
axes.plot(x_plot, y2_plot, alpha=0.15)
axes.scatter(direct_cost, grs_cost, c='r', marker='D')
plt.show()